# Area Assessments and Error Adjustments

In [2]:
import geopandas as gpd
import pandas as pd
import sys
import json
import numpy as np
import rasterio as rs
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

sys.path.append('../../src/')
from evaluation import error_adjustments as err
from analyses import area_assessments as area

%load_ext autoreload
%autoreload 2

In [6]:
## background has higher recall, area is underrepresented
## monoculture has higher recall, area is underrepresented
## agroforestry has higher precision, area is overrepresented
## natural has higher precision, area is overrepresented

In [4]:
def reproject_to_meters(input_raster, output_raster, target_crs='EPSG:3857'):
    
    with rs.open(input_raster) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rs.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rs.band(src, i),
                    destination=rs.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )

In [15]:
prj_districts = gpd.read_file('../../data/epa_districts/Project_Districts.shp')
input_raster = '/Users/jessica.ertel/github/plantation_classifier/tmp/ghana/preds/mosaic/final_2024-10-29.tif'
repj_raster = '/Users/jessica.ertel/github/plantation_classifier/tmp/ghana/preds/mosaic/reprojected_2024-10-29.tif'

# reproject inputs from degrees to meters for area assessment calcs
dst_crs = 'EPSG:3857'
reproject_to_meters(input_raster, repj_raster, dst_crs)
prj_districts = prj_districts.to_crs(dst_crs)

with open("../../data/validation/ci_error_adjustment.json", "r") as f:
    error = json.load(f)

area_assess = area.calculate_adjusted_area(repj_raster, 
                                           prj_districts, 
                                           error, 
                                           '../../data/area_assessments/adj_area_assessment_081825.csv')

In [16]:
area_assess

,District,Area (ha),Background (ha),Monoculture (ha),Agroforestry (ha),Natural (ha),No data
0,Adansi South,77457.0,5958.0,1072.0,53783.0,16628.0,57927.0
1,Asante Akim South,117629.0,4611.0,655.0,97541.0,14791.0,135822.0
2,Assin North,73838.0,3360.0,730.0,66771.0,2964.0,51943.0
3,Atwima Mponua,192078.0,11460.0,5126.0,159426.0,16006.0,168338.0
4,Bawku West,111788.0,92008.0,5.0,16626.0,2381.0,90072.0
5,Bosome Freho,57955.0,3538.0,743.0,41716.0,11938.0,98417.0
6,Builsa South,129292.0,99764.0,1.0,25806.0,3711.0,195523.0
7,Daffiama Bussie Issa,151429.0,114227.0,0.0,17875.0,19311.0,142671.0
8,Juaben Municipal,17479.0,1375.0,550.0,12402.0,3138.0,40548.0
9,Kasena Nankana West,91116.0,68388.0,13.0,20163.0,2529.0,146231.0


In [9]:
# check
old = pd.read_csv( '../../data/area_assessments/adj_area_assessment_042125.csv')

In [10]:
old

,Background,Monoculture,Agroforestry,Natural,No data,district
0,7614,3299,40729,13189,57926.96,Adansi South
1,5893,2016,73866,11732,135822.38,Asante Akim South
2,4294,2245,50564,2351,51943.00,Assin North
3,14647,15769,120730,12695,168338.32,Atwima Mponua
4,117593,16,12590,1889,90072.34,Bawku West
5,4521,2287,31590,9469,98417.49,Bosome Freho
6,127505,3,19542,2944,195523.07,Builsa South
7,145989,1,13536,15317,142670.70,Daffiama Bussie Issa
8,1757,1691,9392,2489,40548.41,Juaben Municipal
9,87405,39,15269,2006,146231.46,Kasena Nankana West


## Create table for publication

Cleans up and structures the results of the error adjusted area assessment table for input into the technical note.

In [7]:
area.area_assessment_table(input_f='adj_area_assessment_081825.csv',
                           output_f='adj_area_assessment_pubfigure_101025.csv',
                           include_summary_row=True)


Total area across all 26 districts (ha):
Monoculture (ha)       13907
Agroforestry (ha)    1562542
Natural (ha)          897523
dtype: int64


,Zone,District,Area (ha),Monoculture (ha),Agroforestry (ha),Natural (ha)
0,north,West Mamprusi Municipal,271590.0,10,32540,2860
1,north,Wa East,446120.0,0,68180,106320
2,north,Talensi,87380.0,0,11100,2640
3,north,Bawku West,111790.0,0,16630,2380
4,north,Sissala West,213860.0,10,41450,8660
5,north,Builsa South,129290.0,0,25810,3710
6,north,Daffiama Bussie Issa,151430.0,0,17880,19310
7,north,Sissala East,530900.0,10,139900,42230
8,north,Kasena Nankana West,91120.0,10,20160,2530
9,north,West Gonja,487310.0,0,117770,145220


### In-text statistics

In [14]:
with open("../../data/validation/ci_error_adjustment.json", "r") as f:
    error = json.load(f)

error

{'overall_accuracy': [0.6605820695807314, 0.027219000892060585],
 'Monoculture': {'recall': [0.8, 0.2],
  'precision': [0.2600760538896132, 0.12453933072577142],
  'adj': 3.0760232940920904},
 'No vegetation': {'recall': [0.899482320888658, 0.02734973167844318],
  'precision': [0.7037827678338109, 0.03833655308379874],
  'adj': 1.2780681227208721},
 'Agroforestry': {'recall': [0.5287270764738249, 0.04639592178972898],
  'precision': [0.6981959088700016, 0.04880965183435426],
  'adj': 0.7572761022469262},
 'Natural': {'recall': [0.4596836237383731, 0.06132923055588704],
  'precision': [0.5795407200450948, 0.06722544910913414],
  'adj': 0.793186065860229}}

In [2]:
368500+343410

711910

## Other // Not using

In [67]:
#calculate total area of raster (just for reference)
src = rs.open(repj_raster)
data = src.read(1)
valid_px = np.sum(data != src.nodata)
pixel_width = src.transform[0]  # X resolution
pixel_height = -src.transform[4]  # Y resolution
px_size = pixel_width * pixel_height
total_area_in_m2 = valid_px * (px_size)
total_area_in_ha = total_area_in_m2 / 10000

# sum all area assessments
# drops nodata column
district_total_area = df.iloc[:, :-2].sum().sum()

round(total_area_in_ha), round(district_total_area)

(5247876, 5247333)

In [70]:
df['ag_area'] = df.agroforestry + df.monoculture
df['nat_area'] = df.natural
df_pubfigure = df[['district', 'ag_area', 'nat_area']]
df_pubfigure

,district,ag_area,nat_area
0,Adansi South,54855.55,16628.07
1,Asante Akim South,98196.58,14791.46
2,Assin North,67500.35,2964.08
3,Atwima Mponua,164552.60,16005.69
4,Bawku West,16630.87,2381.25
5,Bosome Freho,42459.31,11938.37
6,Builsa South,25806.98,3711.35
7,Daffiama Bussie Issa,17875.03,19311.11
8,Juaben Municipal,12951.70,3137.66
9,Kasena Nankana West,20175.93,2528.68
